In [1]:
import pandas as pd
DIR = "../input/moa-ensemble-check/"
mlp = pd.read_csv(DIR+"mlp1840.csv")
tab = pd.read_csv(DIR+"sub_tab1859.csv")
xgb = pd.read_csv(DIR+"xgb1951.csv")

In [2]:
print(mlp.corrwith(xgb).max(), mlp.corrwith(xgb).min(), mlp.corrwith(xgb).mean(), mlp.corrwith(xgb).std())
print(mlp.corrwith(tab).max(), mlp.corrwith(tab).min(), mlp.corrwith(tab).mean(), mlp.corrwith(tab).std())
print(xgb.corrwith(tab).max(), xgb.corrwith(tab).min(), xgb.corrwith(tab).mean(), xgb.corrwith(tab).std())

0.9999999791302244 0.030860694801952644 0.6116583825780203 0.23744212858294525
0.9985353880543508 0.3700866219600505 0.8051177238773994 0.11991313184938018
0.9965936610503259 0.06435090171801035 0.5870370216863928 0.18436435223596784
